In [24]:
import numpy as np
from bigfile import File, Dataset

np.set_printoptions(linewidth=120)

In [221]:
# modified base on `%load read_dmo.py`

# list all the quantities
dm_quantity = ['1/GroupID','1/ID','1/Mass','1/Position','1/Velocity']

fof_quantity = ['FOFGroups/FirstPos','FOFGroups/GroupID','FOFGroups/Imom','FOFGroups/Jmom',
                'FOFGroups/LengthByType','FOFGroups/Mass','FOFGroups/MassByType',
                'FOFGroups/MassCenterPosition','FOFGroups/MassCenterVelocity','FOFGroups/MinID']

snapshot = "/scratch1/06431/yueyingn/dmo-50MPC-fixvel/low-resl/set0/output/PART_004"
pig = File(snapshot)

# redshift
header = pig.open('Header')
redshift = 1. / header.attrs['Time'][0] - 1
boxsize = header.attrs['BoxSize'][0]
Ng = header.attrs['TotNumPart'][1] ** (1/3)
Ng = int(np.rint(Ng))
print("z =", redshift)
print("BoxSize =", boxsize)
print("Ngrid =", Ng)

# Particle information:
#DM_id = pig.open('1/ID')[:] - 1
#DM_pos = pig.open('1/Position')[:]  # in kpc
#DM_vel = pig.open('1/Velocity')[:]

DM = Dataset(pig["1/"], ["ID", "Position", "Velocity"])

z = 6.999999999999998
BoxSize = 50000.0
Ngrid = 440


In [280]:
def pid2subgrid(pid, Ng, sim):
    if Ng == 880:
        hl = "high"
    elif Ng == 440:
        hl = "low"
    else:
        raise
    outpath = f"/scratch1/06589/yinli/dmo-50MPC-fixvel/{hl}-resl/set{sim}/output/PART_004/"
    
    Nsg = Ng // 5
    offset = Nsg // 2
    pid -= 1
    ii, jj, kk = pid // Ng**2, pid // Ng % Ng, pid % Ng
    I, i = ii // offset, ii % offset
    J, j = jj // offset, jj % offset
    K, k = kk // offset, kk % offset
    print(f"I, i = {I}, {i} ({ii}); J, j = {J}, {j} ({jj}); K, k = {K}, {k} ({kk})")
    print(f"First check box {I}{J}{K} at ({i}, {j}, {k})")
    print(np.load(outpath + f"{I}{J}{K}.npy")[i, j, k])
    print("Then there are 7 images which should work fine...")
    
    return ii, jj, kk

def rank2sg(r, Ng, sim, boxsize=50000):
    if Ng == 880:
        hl = "high"
    elif Ng == 440:
        hl = "low"
    else:
        raise
    inpath = f"/scratch1/06431/yueyingn/dmo-50MPC-fixvel/{hl}-resl/set{sim}/output/PART_004/"
    f = File(inpath)
    DM = Dataset(f["1/"], ["ID", "Position", "Velocity"])
    print(DM[r])
    print("--------")
    ii, jj, kk = pid2subgrid(int(DM[r][0]), Ng, sim)
    cellsize = boxsize / Ng
    print("--------")
    print(DM[r][1] - cellsize * np.array([ii, jj, kk]))

In [281]:
Ng = 440
r = np.random.choice(Ng**3)
sim = np.random.choice(5)
rank2sg(r, Ng, sim)
del Ng, sim

(44074768, [25459.79766898, 32947.13720051, 45351.03014262], [ -7.0623536,   1.9144222, -15.952523 ])
--------
I, i = 5, 7 (227); J, j = 6, 25 (289); K, k = 9, 11 (407)
First check box 569 at (7, 25, 11)
[-335.6569     106.22811   -898.96985     -7.0623536    1.9144222  -15.952523 ]
Then there are 7 images which should work fine...
--------
[-335.65687648  106.2281096  -898.96985738]
